In [26]:
import pandas as pd
import numpy as np
import re

# import en_core_web_sm
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
from spacy.matcher import PhraseMatcher
from skillNer.general_params import SKILL_DB
from skillNer.skill_extractor_class import SkillExtractor

# init params of skill extractor
nlp = spacy.load("en_core_web_lg")
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [28]:
df = pd.read_csv('../assistant/data/courssistant.csv')
df.head()

,id,CourseName,University,DifficultyLevel,CourseRating,CourseDescription,Skills,Specialized,Name Adjusted,Description Adjusted,Sequence,Vector,IDF,BERT_Encoded,Course_Name_Encoded
0,1,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,Write a Full Length Feature Film Script In th...,drama comedy peer screenwrite film document re...,Arts and Humanities,write feature length screenplay film television,write full length feature film script course w...,write feature length screenplay film televisio...,[1273.0 2044.0 669.0 359.0 1940.0 2451.0 477.0...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[ 3.09797049e-01 8.47295076e-02 4.00437832e-...,[ 5.41240990e-01 1.87071562e-01 -3.78979407e-...
1,2,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,"By the end of this guided project, you will be...",finance business plan persona user experience ...,Business,business strategy business model canvas analys...,end guide project fluent identifying create bu...,business strategy business model canvas analys...,[275.0 10005.0 732.0 454.0 1789.0 3931.0 663.0...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[-1.62370950e-02 -5.21499366e-02 3.00578505e-...,[-6.51201755e-02 -3.22493613e-01 4.73578013e-...
2,3,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,This course consists of a general presentation...,chemistry physics solar energy film lambda cal...,Physical Science and Engineering,silicon thin film solar cell,course consist general presentation solar cell...,silicon thin film solar cell course consist ge...,[251.0 542.0 1596.0 120.0 324.0 1118.0 453.0 2...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[ 1.11734010e-01 2.74278969e-01 4.72731978e-...,[ 5.20683348e-01 -1.06733255e-01 5.58283515e-...
3,4,Finance for Managers,IESE Business School,Intermediate,4.8,"When it comes to numbers, there is always more...",account receivable dupont analysis analysis ac...,Business,finance manager,come number always meet eye operational financ...,finance manager come number always meet eye op...,[72.0 3326.0 253.0 332.0 415.0 513.0 311.0 259...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[-4.35874946e-02 1.21336013e-01 3.25895667e-...,[-8.81503224e-02 -1.38620347e-01 -2.96908081e-...
4,5,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,In this course you�ll learn how to effectively...,datum analysis select sql database management ...,Information Technology,retrieve datum use single table sql query,course learn effectively retrieve datum relati...,retrieve datum use single table sql query cour...,[421.0 1300.0 588.0 2102.0 3840.0 9546.0 704.0...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[ 6.04762919e-02 3.37209962e-02 4.53004211e-...,[-2.00979590e-01 -8.56521949e-02 3.17591250e-...


In [29]:
df = df.drop(columns=["Name Adjusted", "Description Adjusted", "Sequence", "Vector", "IDF"])
df

,id,CourseName,University,DifficultyLevel,CourseRating,CourseDescription,Skills,Specialized,BERT_Encoded,Course_Name_Encoded
0,1,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,Write a Full Length Feature Film Script In th...,drama comedy peer screenwrite film document re...,Arts and Humanities,[ 3.09797049e-01 8.47295076e-02 4.00437832e-...,[ 5.41240990e-01 1.87071562e-01 -3.78979407e-...
1,2,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,"By the end of this guided project, you will be...",finance business plan persona user experience ...,Business,[-1.62370950e-02 -5.21499366e-02 3.00578505e-...,[-6.51201755e-02 -3.22493613e-01 4.73578013e-...
2,3,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,This course consists of a general presentation...,chemistry physics solar energy film lambda cal...,Physical Science and Engineering,[ 1.11734010e-01 2.74278969e-01 4.72731978e-...,[ 5.20683348e-01 -1.06733255e-01 5.58283515e-...
3,4,Finance for Managers,IESE Business School,Intermediate,4.8,"When it comes to numbers, there is always more...",account receivable dupont analysis analysis ac...,Business,[-4.35874946e-02 1.21336013e-01 3.25895667e-...,[-8.81503224e-02 -1.38620347e-01 -2.96908081e-...
4,5,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,In this course you�ll learn how to effectively...,datum analysis select sql database management ...,Information Technology,[ 6.04762919e-02 3.37209962e-02 4.53004211e-...,[-2.00979590e-01 -8.56521949e-02 3.17591250e-...
...,...,...,...,...,...,...,...,...,...,...
3180,3181,"Capstone: Retrieving, Processing, and Visualiz...",University of Michigan,Beginner,4.6,"In the capstone, students will build a series ...",database syntax analysis web datum visualizati...,Data Science,[-5.06220199e-02 2.78705388e-01 4.92426634e-...,[-2.86793321e-01 9.30177271e-02 1.56947181e-...
3181,3182,Patrick Henry: Forgotten Founder,University of Virginia,Intermediate,4.9,"�Give me liberty, or give me death:� Rememberi...",retirement causality career history book blame...,Arts and Humanities,[-1.05224781e-01 9.72151011e-02 3.67529511e-...,[-2.46693328e-01 1.20946445e-01 -2.40367964e-...
3182,3183,Business intelligence and data analytics: Gene...,Macquarie University,Advanced,4.6,�Megatrends� heavily influence today�s organis...,analytic tableau software business intelligenc...,Business,[ 2.10656263e-02 2.36148432e-01 4.17708278e-...,[-3.80076766e-02 -2.12440696e-02 -5.60713857e-...
3183,3184,Rigid Body Dynamics,Korea Advanced Institute of Science and Techno...,Beginner,4.6,"This course teaches dynamics, one of the basic...",angular mechanical design fluid mechanic frame...,Physical Science and Engineering,[-4.36772071e-02 -2.17585921e-01 6.42734289e-...,[ 2.57868655e-02 -1.76331729e-01 -1.95781782e-...


In [30]:
df.CourseDescription[0]

'Write a Full Length Feature Film Script  In this course, you will write a complete, feature-length screenplay for film or television, be it a serious drama or romantic comedy or anything in between. You�ll learn to break down the creative process into components, and you�ll discover a structured process that allows you to produce a polished and pitch-ready script by the end of the course. Completing this project will increase your confidence in your ideas and abilities, and you�ll feel prepared to pitch your first script and get started on your next. This is a course designed to tap into your creativity and is based in "Active Learning". Most of the actual learning takes place within your own activities - that is, writing! You will learn by doing.  Here is a link to a TRAILER for the course. To view the trailer, please copy and paste the link into your browser. https://vimeo.com/382067900/b78b800dc0  Learner review: "Love the approach Professor Wheeler takes towards this course. It\'s

In [31]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Initialize the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to embed a skill sentence
def embed_skill_sentence(skill_sentence):
    if isinstance(skill_sentence, str):
        embedding = embedding_model.encode([skill_sentence])
        return embedding[0].tolist()
    else:
        return []

# Embed the skill sentences and create a new column for the vectors
df['SkillVectors'] = df['Skills'].apply(embed_skill_sentence)

/opt/anaconda3/envs/rasa/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/opt/anaconda3/envs/rasa/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [32]:
df.head()

,id,CourseName,University,DifficultyLevel,CourseRating,CourseDescription,Skills,Specialized,BERT_Encoded,Course_Name_Encoded,SkillVectors
0,1,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,Write a Full Length Feature Film Script In th...,drama comedy peer screenwrite film document re...,Arts and Humanities,[ 3.09797049e-01 8.47295076e-02 4.00437832e-...,[ 5.41240990e-01 1.87071562e-01 -3.78979407e-...,"[-0.003577765543013811, 0.017490142956376076, ..."
1,2,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,"By the end of this guided project, you will be...",finance business plan persona user experience ...,Business,[-1.62370950e-02 -5.21499366e-02 3.00578505e-...,[-6.51201755e-02 -3.22493613e-01 4.73578013e-...,"[0.03897735849022865, 0.06576909124851227, -0...."
2,3,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,This course consists of a general presentation...,chemistry physics solar energy film lambda cal...,Physical Science and Engineering,[ 1.11734010e-01 2.74278969e-01 4.72731978e-...,[ 5.20683348e-01 -1.06733255e-01 5.58283515e-...,"[-0.08079390972852707, 0.09204946458339691, 0...."
3,4,Finance for Managers,IESE Business School,Intermediate,4.8,"When it comes to numbers, there is always more...",account receivable dupont analysis analysis ac...,Business,[-4.35874946e-02 1.21336013e-01 3.25895667e-...,[-8.81503224e-02 -1.38620347e-01 -2.96908081e-...,"[0.0005424825940281153, 0.028139270842075348, ..."
4,5,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,In this course you�ll learn how to effectively...,datum analysis select sql database management ...,Information Technology,[ 6.04762919e-02 3.37209962e-02 4.53004211e-...,[-2.00979590e-01 -8.56521949e-02 3.17591250e-...,"[0.00521202152594924, -0.0020099657122045755, ..."


In [33]:
df.to_csv("../assistant/data/courssistant_main.csv", index=False)